In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import feature_spaces
import pandas as pd
from sklearn.preprocessing import StandardScaler
from typing import List
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import encoding_utils, feature_spaces
import pickle as pkl
from collections import defaultdict
from datasets import Dataset, DatasetDict
from feature_spaces import *
NUM_VOXELS = 250

In [2]:
subj = 'UTS03'

# select top_idxs
save_dir = '/home/chansingh/mntv1/deep-fMRI/results/encoding/bert-10__ndel=4/UTS03'
corrs_val = np.load(join(save_dir, 'corrs.npz'))['arr_0']
top_idxs = np.argsort(corrs_val)[::-1][:NUM_VOXELS]
print(corrs_val[top_idxs][:5])

[0.76296981 0.74258237 0.72107898 0.71616266 0.71508206]


# Visualize top voxels and which ROIs they are in

In [3]:
import cortex
viz_cortex = __import__('03_viz_cortex')

In [ ]:
# visualize after masking anything that isn't in the top_idxs
corrs = corrs_val

for i in range(len(corrs)):
    if i not in top_idxs:
        corrs[i] = 0
viz_cortex.quickshow(corrs)

In [6]:
subject = "UTS03"
xfm = "UTS03_auto"

# Get the map of which voxels are inside of our ROI
index_volume, index_keys = cortex.utils.get_roi_masks(
    subject, xfm,
    roi_list=None, # Default (None) gives all available ROIs in overlays.svg
    gm_sampler='cortical-conservative', # Select only voxels mostly within cortex
    split_lr=True, # Separate left/right ROIs (this occurs anyway with index volumes)
    threshold=0.9, # convert probability values to boolean mask for each ROI
    return_dict=False # return index volume, not dict of masks
)

# Count how many of the top_voxels are in each ROI
roi_counter = defaultdict(list)
roi_dict = {}
for k in tqdm(index_keys):
    roi_verts = cortex.get_roi_verts('UTS03', k)
    for val in roi_verts.values():
        for i, top_idx in enumerate(top_idxs):
            if top_idx in val:
                roi_counter[k].append(top_idx)
                roi_dict[i] = k
counts = {k: len(v) for k, v in roi_counter.items()}
sorted(counts.items(), key=lambda s: s[1], reverse=True)

Found 100.00% of S1F
Found 100.00% of M1F
Found 100.00% of TOS
Found 100.00% of FEF
Found 100.00% of S1M
Found 100.00% of sPMv
Found 100.00% of EBA
Found 100.00% of SMFA
Found 100.00% of IPS 
Found 100.00% of PMvh
Found 100.00% of FBA
Found 100.00% of S2H
Found 100.00% of S2F
Found 100.00% of FFA
Found 100.00% of OFA
Found 100.00% of SMHA
Found 100.00% of RSC
Found 100.00% of Broca
Found 100.00% of S2M
Found 100.00% of hMT
Found 100.00% of AC
Found 100.00% of IFSFP
Found 100.00% of PPA
Found 100.00% of M1M
Found 100.00% of S1H
Found 100.00% of M1H
Cutting 47 overlapping voxels (should be < ~50)


100%|██████████| 26/26 [00:47<00:00,  1.83s/it]


[('IPS ', 12),
 ('EBA', 9),
 ('AC', 8),
 ('sPMv', 6),
 ('S1H', 6),
 ('RSC', 5),
 ('S1F', 4),
 ('TOS', 4),
 ('FEF', 4),
 ('SMFA', 4),
 ('OFA', 4),
 ('S2H', 3),
 ('SMHA', 3),
 ('PPA', 3),
 ('S1M', 2),
 ('FFA', 2),
 ('S2M', 2),
 ('S2F', 1),
 ('hMT', 1),
 ('M1M', 1),
 ('M1H', 1)]

In [7]:
pkl.dump(roi_dict, open('roi_dict.pkl', 'wb'))

In [8]:
!cp roi_dict.pkl ../../mprompt/mprompt/modules/fmri/